In [1]:
# Boter kaas en eieren bot met reinforcement learning volgens het monte-carlo principe.
# Author: Ole Timmers - S1115222

In [2]:
# Allereerst importeren we het nodige:
import numpy as np
import random

In [3]:
# Testbord - dit is hoe een bord zal worden aangemaakt.
test_bord = np.matrix([[' ',' ','o'],[' ',' ','o'],['x','x',' ']])

In [4]:
# Bereken de unieke code waarin een state kan worden vastgelegd.
# Dit gaan we gebruiken als zoekcode om de gemiddelde win-ratio vast te leggen
# in een dictionary.
# returns int

def berekenstate(bordToCalc):
    k = 0
    h = 0
    for i in range(len(bordToCalc)):
      for j in range(len(bordToCalc)):
        if bordToCalc[i,j]   == ' ':
          v = 0
        elif bordToCalc[i,j] == 'x':
          v = 1
        elif bordToCalc[i,j] == 'o':
          v = 2
        h += (3**k) * v
        k += 1
    return h

In [5]:
# Hiermee kunnen we het bord printen in de console.
def draw_board(bordToDraw):
    for i in range(len(bordToDraw)):
      print("-------------")
      for j in range(len(bordToDraw)):
        print("  ", end="")
        if bordToDraw[i,j] == 'x':
          print("x ", end="")
        elif bordToDraw[i,j] == 'o':
          print("o ", end="")
        else:
          print("  ", end="")
      print("")
    print("-------------")

In [6]:
#Testje
draw_board(test_bord)

-------------
          o 
-------------
          o 
-------------
  x   x     
-------------


In [7]:
#Test berekenstate
state = berekenstate(test_bord)
print(state)

3420


In [8]:
# Krijg alle mogelijke bewegingen te zien
# Note: Je ziet dat ik allerlei verschillende variabel-namen gebruik voor het bord.
#       Dat is in principe niet nodig natuurlijk, toch deed ik het omdat ik verschillende variabelen niet uit elkaar kon halen. XD
#       Dus in principe kan je overal 'bord' zetten.
def getMogelijkeBewegingen(bordjee):
    moves = []
    for i in range(len(bordjee)):
      for j in range(len(bordjee)):
        if bordjee[i,j] == " ":
                moves.append((i, j))
    return moves

In [9]:
#Test getMogelijkeBewegingen
print(getMogelijkeBewegingen(test_bord))

[(0, 0), (0, 1), (1, 0), (1, 1), (2, 2)]


In [10]:
# getWinner(0), functie die checkt of er een winner is. 
# Bij 1 of 2 is er een winnaar.
# Bij 0 gelijkspel
# Bij -1 zijn er nog bewegingen te maken.
def getWinner(bord):
    candidate = ' '
    won = 0
    
    # Eerst bekijken we rij voor rij
    for i in range(len(bord)):
        candidate = ' '
        for j in range(len(bord)):
            
            # Wanneer er een gat is in een rij, is er sws geen winnaar.
            if bord[i,j] == ' ':
                break
            
            # Bekijk wat er in het eerste vakje staat.
            # Want wanneer dat vakje is gevuld, moeten de rest v/d vakjes
            # hetzelfde symbool hebben.
            if candidate == ' ':
                candidate = bord[i,j]
            
            # Break op wanneer symbolen niet overeen komen.
            if candidate != bord[i,j]:
                break
            elif j == len(bord) - 1:
                # Wanneer we alle drie de vakjes hebben gehad
                # en er is niet gebreaked. Dan hebben we een winnaar.
                if candidate == 'x':
                    won = 1
                else:
                    won = 2
    
    if won > 0:
        return won
    
    # Nu doen we hetzelfde voor de kolommen.
    for j in range(len(bord)):
        candidate = ' '
        for i in range(len(bord)):
            
            # Geen gaten in de kolom - check
            if bord[i,j] == ' ':
                break
            
            # Eerste symbool opvangen
            if candidate == ' ':
                candidate = bord[i,j]
            
            # Controleren of ze allemaal gelijk zijn.
            if candidate != bord[i,j]:
                break
            elif i == len(bord) - 1:
                # We hebben een winnaar.
                if candidate == 'x':
                    won = 1
                else:
                    won = 2
    
    if won > 0:
        return won
    
    # Nu gaan we de diagonalen controleren.
    # (0,0), (1,1) & (2,2)
    candidate = ' '
    for i in range(len(bord)):
        if bord[i, i] == ' ':
            break
        if candidate == ' ':
            candidate = bord[i,i]
        if candidate != bord[i,i]:
            break
        elif i == len(bord) - 1:
            # We hebben een winnaar.
            if candidate == ' ':
                won = 1
            else:
                won = 2
    
    if won > 0:
        return won
    
    # (0,2), (1,1) (2,0)
    candidate = ""
    for i in range(len(bord)):
        if bord[i,2 - i] == ' ':
            break
        if candidate == ' ':
            candidate = bord[i , 2 - i]
        if candidate != bord[i , 2 - i]:
            break
        elif i == len(bord) - 1:
            # We hebben een winnaar.
            if candidate == 'x':
                won = 1
            else:
                won = 2
    
    if won > 0:
        return won
    
    # Nog steeds geen winnaar?
    if (len(getMogelijkeBewegingen(bord)) == 0):
        # It's a draw
        return 0
    else:
        # Nog bewegingen te maken
        return -1

In [11]:
#Test getwinner
print(getWinner(test_bord))

-1


In [12]:
# getRandomMove()
# Om een 'domme' bot te simuleren, maken wij gebruik van getRandom om 
# een willekeurige beweging uit te voeren.
def getRandomMove(bord):
    mogelijkeBewegingen = getMogelijkeBewegingen(bord)
    return random.choice(mogelijkeBewegingen)

In [13]:
# Simuleer de game - random vs random
# hiermee simuleer ik de game waar beide players random zullen gaan kiezen.
# De stappen van p1 (x) zet ik als states in de historie
# Deze zal ik verolgens returnen waarna deze in een dictionary kunnen worden gezet i.c.m. winst/verlies.

# Dit was een eerste stap in het machine learning gedeelte, omdat ik hier nog niet het reinforcement learning-gedeelte toepas.
# Later heb ik ook nog een functie waar 'ie wel ook aan reinfocement learning doet.
def simulateGameWithRandomMoves():
    historieVanStatesP1 = []
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        
        # Kies random move
        move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesP1.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            playerToMove = 'x'
            
    return (historieVanStatesP1, getWinner(currentbord))

In [14]:
# Test winratio - random vs random
# Hier test ik bovenstaande functies in een for loop en kijk ik wat de resultaten zijn. 
wins = 0
losses = 0
draws = 0

for i in range(10000):
    resultOfGame = simulateGameWithRandomMoves()
    uitslag = resultOfGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
        
print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

wins: 4183
losses: 3426
draws: 2391


In [15]:
# berekenNieuwGemiddelde(currentGemiddelde, numberToBeAdded, currentCount)
# Met deze functie pak ik het oude gemiddelde(currentgemiddelde) en het getal
# over hoeveel keer dat gemiddelde is genomen.
# Met die twee gegevens en het nieuwe resultaat, maak ik een nieuwe gemiddelde aan.

# Voor de duidelijkheid: ik bereken hier het gemiddelde van een bepaalde state die ik heb uitgevoerd nadat een potje is afgelopen.
def berekenNieuwGemiddelde(currentGemiddelde, numberToBeAdded, currentCount):
    totalNumber = currentGemiddelde * currentCount
    if numberToBeAdded == 2:
        totalNumber += 2
    elif numberToBeAdded == 1:
        totalNumber += 0
    else:
        totalNumber += 1
    newCount = currentCount + 1
    return ((totalNumber / newCount), newCount)

In [16]:
# Nadat een game is geweest, slaan wij alle states op die x heeft doorgevoerd met het resultaat van de game.
# In een dictionary, die aan deze functie wordt meegegeven, gaan wij vervolgens voor iedere state 
# dit resultaat aan de gemiddelde win-ratio toevoegen en de nieuwe ratio daarna berekenen.
# Het daadwerkelijk berekenen van de ratio (het gemiddelde) doen we in de functie hierboven.

# In deze functie itereren wij door alle states heen, en voegen/passen wij deze toe/aan in de dictionary.
def addToDictionary(dictio, historie):
    states = historie[0]
    resultVanGame = historie[1]
    resultToBeAdded = 0
    # Bij verlies niks toevoegen aan gemiddelde
    if (resultVanGame == 2):
        resultToBeAdded = 0
    elif (resultVanGame == 1): # Bij winst 2
        resultToBeAdded = 2
    else:
        resultToBeAdded = 1 # Bij gelijkspel 1

    for state in states:
        # Als de entry in de dictionary nog niet bestaat, maak 'm aan.
        if not (str(state) in dictio.keys()):
            dictio[str(state)] = berekenNieuwGemiddelde(1.0, resultToBeAdded, 1)
        else:
            dictio[str(state)] = berekenNieuwGemiddelde(dictio[str(state)][0], resultToBeAdded, dictio[str(state)][1])
        
    return dictio
    

In [17]:
# Hier komt het reinforcment learning gedeelte echt van pas.
# Bij activatie van deze functie, kijkt de code naar alle mogelijkheden.
# Voor iedere mogelijkheid, maakt hij een bord van hoe het bord er uit moet zien en converteert dit vervolgens naar een statenr.
# Dit statenr is onze zoekcode binnen de dictionary, en hiermee kunnen we de winstratio ophalen.
# De state met de hoogste winstratio wordt vervolgens als beste move gekozen.
def berekenBesteMove(bordje, dictio, place = 'x'):
    mogelijkeBewegingen = getMogelijkeBewegingen(bordje)
    bestMove = (mogelijkeBewegingen[0], 0) # beginnen bij 0
    for mogelijkheid in mogelijkeBewegingen:
        # We nemen het bord over
        bordToCalculateIn = np.matrix([[bordje[0,0],bordje[0,1],bordje[0,2]],[bordje[1,0],bordje[1,1],bordje[1,2]],[bordje[2,0],bordje[2,1],bordje[2,2]]])
        # en voeren de move uit om naar die state te komen.
        bordToCalculateIn[mogelijkheid[0], mogelijkheid[1]] = place
        # We maken het statenr
        statenr = berekenstate(bordToCalculateIn)
        # en zoeken deze op in onze dictionary
        if (not (str(statenr) in dictio.keys())):
            if (bestMove[1] < 1):
                bestMove = (mogelijkheid, 1)
        else:
            if (bestMove[1] < dictio[str(statenr)][0]):
                bestMove = (mogelijkheid, dictio[str(statenr)][0])
    return bestMove

In [18]:
# Simuleer de game - reinfocement vs random
# Zelfde als de andere simuleergame, echter zal player x hier volgens reinforcment learning de juiste keuzes maken.
# Met de paramter y kunnen wij de ratio verkennen/uitbuiten vaststellen. 
# Hoe lager hoe meer uitbuiting, hoe hoger hoe meer verkennen.

def simuleerGameWithDictionary(dictio, y, printBoard = False):
    historieVanStatesP1 = []
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        # Kies random move
        if playerToMove == 'x':
            # alleen deze regels zijn anders dan de vorige functie.
            if (random.random() > y):
                move = berekenBesteMove(currentbord, dictio)[0]
            else:
                move = getRandomMove(currentbord)
                
        else:
            move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.
        if printBoard:
            print('===========')
            draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesP1.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            playerToMove = 'x'
            
    return (historieVanStatesP1, getWinner(currentbord))

In [19]:
dictionaryToTestWith = {}

In [20]:
# HET TESTEN van de reinforcment learning
# Dit was nogal een gekloot tussen verkennen/uitbuiten - wat is nou beter?? - maar uiteindelijk heb ik voor het volgende gekozen:
# Eerst voeg ik aan de dictionary 150 000 games toe waarbij voor 90% aan verkennen wordt gedaan.
# Daarna doe ik het andersom, 90% uitbuiten en 10% verkennen.
# Als laatste doe ik 10 000 x een game met 100% uitbuiten om de resultaten te zien.

for i in range(150000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0.9)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

    
for i in range(150000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0.1)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

wins = 0
losses = 0
draws = 0

# De final test....

for i in range(10000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0, False)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

wins: 9411
losses: 555
draws: 34


In [21]:
# Oké gaaf, de bot heeft nu een 19/20 winstratio. Maar het wordt nog niet enorm uitgedaagd tegen de random bot.
# Wat ik nu wil is dat we beide kanten gaan trainen, zo worden ze echt competatief is mijn hypothese.
#
# Allereerst maak ik een nieuwe simmuleer functie waarbij ik beide machines tegen elkaar kan laten trainen.
# Vervolgens initialiseer ik de dictionary opnieuw + ik maak een nieuwe dictionary voor speler 'o'

In [22]:
# De nieuwe functie die we willen hebben.
# We hebben nu twee verschillende dictionaries, daarom hebben we nu ook de parameter 'dictios'.
def simuleerGameWithDictionaries(dictios, y, printBoard = False):
    historieVanStatesPX = []
    historieVanStatesPO = [] # We houden nu ook de historie van speler 'o' bij. 
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        # Kies random move
        if playerToMove == 'x':
            # alleen deze regels zijn anders dan de vorige functie.
            if (random.random() > y):
                move = berekenBesteMove(currentbord, dictios[0])[0] # dictios[0] is de dictionary van speler 'x'
            else:
                move = getRandomMove(currentbord)
                
        else:
            if (random.random() > y): # Dit is toegevoegd om ook speler 'o' te laten trainen.
                move = berekenBesteMove(currentbord, dictios[1], 'o')[0] # dictios[1] is de dictionary van speler 'o'
            else:
                move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.
        if printBoard:
            print('===========')
            draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesPX.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            historieVanStatesPO.append(berekenstate(currentbord))
            playerToMove = 'x'
    
    # end of while-loop
            
    # Nu gaan we beide histories returnen om deze te trainen.
    # Om de addToDictionary functie te behouden (deze is nu gebasseerd op de 1/2 voor speler 'x'),
    # Moeten we voor speler 'o' de eventuele 1 & 2 omdraaien, 
    # om zo de win/loss van speler 'o' in zijn eigen dictio goed te zetten.
    # Bij 0 laten we dit gewoon staan - geen winnaar is nog steeds 0.
    
    resultForPlayerX = getWinner(currentbord) # dit is dus het resultaat voor de dictionary van speler 'x'
    resultForPlayerO = 0 # gelijk spel is default
    if (resultForPlayerX == 2):
        resultForPlayerO = 1
    elif (resultForPlayerX == 1):
        resultForPlayerO = 2
    
    # De data wordt nog steeds verzameld in tuples, maar nu krijgen we een tuple in een tuple
    # De eerste tuple bevat data van speler 'x', de tweede van 'o'
        
    dataPlayerX = (historieVanStatesPX, resultForPlayerX)
    dataPlayerO = (historieVanStatesPO, resultForPlayerO)
                
    return (dataPlayerX, dataPlayerO)

In [23]:
# Hier definieer ik beide dictionaries die we gaan gebruiken.
dictionaryPlayerX = {}
dictionaryPlayerO = {}

In [24]:
# We gaan weer opnieuw testen, dit x worden beide spelers getraind.
# Ik gebruik dezelfde stappen als dat ik voorheen deed met de vorige trainingsessie.
# Alleen nu gaan we beide dictionaries vullen en vervolgens invoeren in de simmuleer-functie.

for i in range(150000):
    dataVanDeGame = simuleerGameWithDictionaries((dictionaryPlayerX, dictionaryPlayerO), 0.9)
    historieVanGamePX = dataVanDeGame[0]
    historieVanGamePO = dataVanDeGame[1]
    uitslag = historieVanGamePX[1]

    dictionaryPlayerX = addToDictionary(dictionaryPlayerX, historieVanGamePX)
    dictionaryPlayerO = addToDictionary(dictionaryPlayerO, historieVanGamePO)

    
for i in range(150000):
    dataVanDeGame = simuleerGameWithDictionaries((dictionaryPlayerX, dictionaryPlayerO), 0.1)
    historieVanGamePX = dataVanDeGame[0]
    historieVanGamePO = dataVanDeGame[1]
    uitslag = historieVanGamePX[1]

    dictionaryPlayerX = addToDictionary(dictionaryPlayerX, historieVanGamePX)
    dictionaryPlayerO = addToDictionary(dictionaryPlayerO, historieVanGamePO)

playerXWins = 0
playerOWins = 0
draws = 0

# De final test....

for i in range(10000):
    dataVanDeGame = simuleerGameWithDictionaries((dictionaryPlayerX, dictionaryPlayerO), 1, False)
    
    historieVanGamePX = dataVanDeGame[0]
    historieVanGamePO = dataVanDeGame[1]
    uitslag = historieVanGamePX[1]
    
    if (uitslag == 1):
        playerXWins += 1
    elif (uitslag == 2):
        playerOWins += 1
    else:
        draws += 1

    dictionaryPlayerX = addToDictionary(dictionaryPlayerX, historieVanGamePX)
    dictionaryPlayerO = addToDictionary(dictionaryPlayerO, historieVanGamePO)  

print('Player X wins: ' + str(playerXWins))
print('Player O wins: ' + str(playerOWins))
print('draws: ' + str(draws))

Player X wins: 4219
Player O wins: 3301
draws: 2480


In [25]:
# Laten we het verschil met de oude trainingmethode is bekijken, wordt deze beter als wij bot vs bot doen?

wins = 0
losses = 0
draws = 0
playerXWins = 0
playerOWins = 0
draws2 = 0

# Test met oude functie en oude dictionary

for i in range(10000):
    historieVanGame = simuleerGameWithDictionary(dictionaryToTestWith, 0, False)
    uitslag = historieVanGame[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
    dictionaryToTestWith = addToDictionary(dictionaryToTestWith, historieVanGame)

print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

# Test met oude functie en nieuwe dictionary

for i in range(10000):
    dataVanDeGame = simuleerGameWithDictionary(dictionaryPlayerX, 1)
    
    uitslag = dataVanDeGame[1]
    
    if (uitslag == 1):
        playerXWins += 1
    elif (uitslag == 2):
        playerOWins += 1
    else:
        draws2 += 1

print('Player X wins: ' + str(playerXWins))
print('Player O wins: ' + str(playerOWins))
print('draws: ' + str(draws2))

wins: 9454
losses: 528
draws: 18
Player X wins: 4349
Player O wins: 3287
draws: 2364


In [26]:
# 2 getrainde bots tegen elkaar geeft nog niet echt een duidelijk resultaat.
# Kijken wat er gebeurt als we een extra getrainde bot (bot vs bot) tegen een random bot laten spelen.

def simuleerGameWithDictionariesVsRandom(dictios, y, printBoard = False, randomPlayer = 'o'):
    historieVanStatesPX = []
    historieVanStatesPO = [] # We houden nu ook de historie van speler 'o' bij. 
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    xIsNotRandom = 'x' != randomPlayer
    oIsNotRandom = 'o' != randomPlayer
    
    while getWinner(currentbord) == -1:
        # Kies random move
        if playerToMove == 'x':
            if (random.random() > y and xIsNotRandom):
                move = berekenBesteMove(currentbord, dictios[0])[0]
            else:
                move = getRandomMove(currentbord)
                
        else:
            if (random.random() > y and oIsNotRandom): 
                move = berekenBesteMove(currentbord, dictios[1], 'o')[0]
            else:
                move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.
        if printBoard:
            print('===========')
            draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesPX.append(berekenstate(currentbord))
            # switch player
            playerToMove = 'o'
        else:
            historieVanStatesPO.append(berekenstate(currentbord))
            playerToMove = 'x'
    
    # end of while-loop
            
    # Nu gaan we beide histories returnen om deze te trainen.
    # Om de addToDictionary functie te behouden (deze is nu gebasseerd op de 1/2 voor speler 'x'),
    # Moeten we voor speler 'o' de eventuele 1 & 2 omdraaien, 
    # om zo de win/loss van speler 'o' in zijn eigen dictio goed te zetten.
    # Bij 0 laten we dit gewoon staan - geen winnaar is nog steeds 0.
    
    resultForPlayerX = getWinner(currentbord) # dit is dus het resultaat voor de dictionary van speler 'x'
    resultForPlayerO = 0 # gelijk spel is default
    if (resultForPlayerX == 2):
        resultForPlayerO = 1
    elif (resultForPlayerX == 1):
        resultForPlayerO = 2
    
    # De data wordt nog steeds verzameld in tuples, maar nu krijgen we een tuple in een tuple
    # De eerste tuple bevat data van speler 'x', de tweede van 'o'
        
    dataPlayerX = (historieVanStatesPX, resultForPlayerX)
    dataPlayerO = (historieVanStatesPO, resultForPlayerO)
                
    return (dataPlayerX, dataPlayerO)

In [27]:
# De final test....

playerXWins = 0
playerOWins = 0
draws = 0

for i in range(1000):
    dataVanDeGame = simuleerGameWithDictionariesVsRandom((dictionaryPlayerX, dictionaryPlayerO), 1, False, 'x')
    
    historieVanGamePX = dataVanDeGame[0]
    historieVanGamePO = dataVanDeGame[1]
    uitslag = historieVanGamePX[1]
    
    if (uitslag == 1):
        playerXWins += 1
    elif (uitslag == 2):
        playerOWins += 1
    else:
        draws += 1

    dictionaryPlayerX = addToDictionary(dictionaryPlayerX, historieVanGamePX)
    dictionaryPlayerO = addToDictionary(dictionaryPlayerO, historieVanGamePO)

print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))

wins: 9454
losses: 528
draws: 220


In [28]:
# Bots tegen elkaar laten trainen heeft nog niet echt veel goeds gedaan helaas.
# Wat ik nu nog ga doen is player o ook op de oude manier laten trainen.
# Zo hebben we een getrainde player x en een getrainde player o.
# Deze wil ik vervolgens tegen de echte speler laten spelen.
# Wanneer je dan als eerste begint, speel je tegen o, en andersom speel je tegen x. 
# Bot x en o worden dus een gezamenlijke bot.

In [29]:
def simuleerGameWithDictionaryForPlayerO(dictio, y, printBoard = False):
    historieVanStatesP2 = []
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        # Kies random move
        if playerToMove == 'o':
            if (random.random() > y):
                move = berekenBesteMove(currentbord, dictio, 'o')[0]
            else:
                move = getRandomMove(currentbord)
                
        else:
            move = getRandomMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.
        if printBoard:
            print('===========')
            draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # switch player
            playerToMove = 'o'
        else:
            # Voeg de nieuwe status van het bord (door speler 1) aan de historie
            historieVanStatesP2.append(berekenstate(currentbord))
            playerToMove = 'x'
    
    initialResult = getWinner(currentbord) # dit is dus het resultaat voor de dictionary van speler 'x'
    actualResult = 0 # gelijk spel is default
    if (initialResult == 2):
        actualResult = 1
    elif (initialResult == 1):
        actualResult = 2
        
            
    return (historieVanStatesP2, actualResult)

In [30]:
finalDictionaryPlayerX = {}
finalDictionaryPlayerO = {}

In [31]:
# Hieronder weer het bekende train-stukje.
# Dit keer train ik de player x én player o apart van elkaar.

In [32]:
for i in range(150000):
    finalHistoriePlayerX = simuleerGameWithDictionary(finalDictionaryPlayerX, 0.9)
    finalHistoriePlayerO = simuleerGameWithDictionaryForPlayerO(finalDictionaryPlayerO, 0.9)
    
    finalDictionaryPlayerX = addToDictionary(finalDictionaryPlayerX, finalHistoriePlayerX)
    finalDictionaryPlayerO = addToDictionary(finalDictionaryPlayerO, finalHistoriePlayerO)
    

    
for i in range(150000):
    finalHistoriePlayerX = simuleerGameWithDictionary(finalDictionaryPlayerX, 0.1)
    finalHistoriePlayerO = simuleerGameWithDictionaryForPlayerO(finalDictionaryPlayerO, 0.1)

    finalDictionaryPlayerX = addToDictionary(finalDictionaryPlayerX, finalHistoriePlayerX)
    finalDictionaryPlayerO = addToDictionary(finalDictionaryPlayerO, finalHistoriePlayerO)
    
wins = 0
losses = 0
draws = 0

var1 = 0;
var2 = 0;
draw2 = 0;

for i in range(10000):
    finalHistoriePlayerX = simuleerGameWithDictionary(finalDictionaryPlayerX, 0, False)
    finalHistoriePlayerO = simuleerGameWithDictionaryForPlayerO(finalDictionaryPlayerO, 0, False)

    finalDictionaryPlayerX = addToDictionary(finalDictionaryPlayerX, finalHistoriePlayerX)
    finalDictionaryPlayerO = addToDictionary(finalDictionaryPlayerO, finalHistoriePlayerO)

    uitslag = finalHistoriePlayerX[1]
    if (uitslag == 1):
        wins += 1
    elif (uitslag == 2):
        losses += 1
    else:
        draws += 1
        
    uitslag2 = finalHistoriePlayerO[1]
    if (uitslag2 == 1):
        var1 += 1
    elif (uitslag2 == 2):
        var2 += 1
    else:
        draw2 += 1

print('wins: ' + str(wins))
print('losses: ' + str(losses))
print('draws: ' + str(draws))
print('wins: ' + str(var1))
print('losses: ' + str(var2))
print('draws: ' + str(draw2))



wins: 9453
losses: 547
draws: 0
wins: 8611
losses: 1389
draws: 0


In [33]:
# Nog een extra testje om te kijken of alles wel goed gaat.
for i in range(10):
    finalHistoriePlayerO = simuleerGameWithDictionaryForPlayerO(finalDictionaryPlayerO, 0, True)

    finalDictionaryPlayerO = addToDictionary(finalDictionaryPlayerO, finalHistoriePlayerO)

-------------
          x 
-------------
            
-------------
            
-------------
-------------
          x 
-------------
      o     
-------------
            
-------------
-------------
          x 
-------------
      o     
-------------
  x         
-------------
-------------
  o       x 
-------------
      o     
-------------
  x         
-------------
-------------
  o       x 
-------------
  x   o     
-------------
  x         
-------------
-------------
  o       x 
-------------
  x   o     
-------------
  x       o 
-------------
-------------
            
-------------
            
-------------
          x 
-------------
-------------
      o     
-------------
            
-------------
          x 
-------------
-------------
  x   o     
-------------
            
-------------
          x 
-------------
-------------
  x   o     
-------------
            
-------------
      o   x 
-------------
-------------
  x   o     
-------------
         

In [34]:
# Oké nice! Zo te zien hebben we nu echt twee getrainde bots.
# Ik ga nu functies maken zodat je zelf kan spelen tegen een van de twee getrainde bots.

In [35]:
# Het plaatsen van een move - interactief
def placeMove(huidigBordje):
    for i in range(len(huidigBordje)):
      print("---------------")
      for j in range(len(huidigBordje)):
        if (j != 0):
            print("| ", end="")
        else:
            print(" ", end="")
            
        if huidigBordje[i,j] == 'x':
          print(" x ", end="")
        elif huidigBordje[i,j] == 'o':
          print(" o ", end="")
        else:
          print(str(i) + "," + str(j), end="")
      print("")
    print("---------------")
    
    mogelijkeBewegingen = getMogelijkeBewegingen(huidigBordje)
    validInput = False
    while not validInput:
        print('Type coördinaat in (bijv. "2,1"): ')
        coo = input()
        try:
            waardes = coo.split(',')
            x = int(waardes[0])
            y = int(waardes[1])
            tupletje = (x,y)
            if tupletje in mogelijkeBewegingen:
                
                return tupletje
        
        except:
            "Onjuiste invoer, probeer opnieuw!"
        
        
    
    

In [36]:
# Test placeMove
placeMove(test_bord)

---------------
 0,0| 0,1|  o 
---------------
 1,0| 1,1|  o 
---------------
  x |  x | 2,2
---------------
Type coördinaat in (bijv. "2,1"): 
2,2


(2, 2)

In [37]:
# Dit keer hoeven we geen historie meer terug te geven, het gaat nu echt alleen om het spelen zelf.
def playGame(dictio, humanPlayer):
    currentbord = np.matrix([[' ',' ',' '],[' ',' ',' '],[' ',' ',' ']])
    playerToMove = 'x'
    
    while getWinner(currentbord) == -1:
        if playerToMove == 'o':
            if (humanPlayer != 'o'):
                move = berekenBesteMove(currentbord, dictio, 'o')[0]
            else:
                move = placeMove(currentbord)
                
        else:
            if (humanPlayer != 'x'):
                move = berekenBesteMove(currentbord, dictio)[0]
            else:
                move = placeMove(currentbord)
        
        # Doe beweging
        currentbord[move[0], move[1]] = playerToMove
        
        # Mochten we het resultaat van de move willen printen, pas de parameter aan.

        print('==============')
        draw_board(currentbord)
        
        
        if playerToMove == 'x':
            # switch player
            playerToMove = 'o'
        else:
            playerToMove = 'x'
    
    initialResult = getWinner(currentbord) # dit is dus het resultaat voor de dictionary van speler 'x'
    if (initialResult == 2):
        print("Speler o heeft gewonnen!")
    elif (initialResult == 1):
        print("Speler x heeft gewonnen!")
        
            

In [38]:
def initGame():
    while True:
        print("Welke speler wil je zijn?")
        playerInput = input()
        if playerInput == 'x':
            playGame(finalDictionaryPlayerO, 'x')
        elif playerInput == 'o':
            playGame(finalDictionaryPlayerX, 'o')

In [39]:
# Final Product!!!
initGame()

Welke speler wil je zijn?
o
-------------
          x 
-------------
            
-------------
            
-------------
---------------
 0,0| 0,1|  x 
---------------
 1,0| 1,1| 1,2
---------------
 2,0| 2,1| 2,2
---------------
Type coördinaat in (bijv. "2,1"): 
1,2
-------------
          x 
-------------
          o 
-------------
            
-------------
-------------
  x       x 
-------------
          o 
-------------
            
-------------
---------------
  x | 0,1|  x 
---------------
 1,0| 1,1|  o 
---------------
 2,0| 2,1| 2,2
---------------
Type coördinaat in (bijv. "2,1"): 
0,1
-------------
  x   o   x 
-------------
          o 
-------------
            
-------------
-------------
  x   o   x 
-------------
          o 
-------------
  x         
-------------
---------------
  x |  o |  x 
---------------
 1,0| 1,1|  o 
---------------
  x | 2,1| 2,2
---------------
Type coördinaat in (bijv. "2,1"): 
1,0
-------------
  x   o   x 
-------------
  o       o 

KeyboardInterrupt: Interrupted by user